In [4]:
%load_ext autoreload
%autoreload 2

import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn

from ads_analytics import (
    get_facebook_ads_campaign_metrics,
    get_google_ads_campaign_metrics,
)
from google_analytics import get_landing_page_report
from hubspot_conversions import get_hubspot_conversions
from sales import get_sales_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def get_conversions():
    hs = get_hubspot_conversions()
    sales = get_sales_data()
    fb = get_facebook_ads_campaign_metrics()
    ga = get_google_ads_campaign_metrics()
    lp = get_landing_page_report()
    conversions = pd.concat([hs, sales, fb, ga, lp])
    return conversions


In [7]:
conversions = get_conversions()
conversions

[2025-03-06 18:41:08 - INFO] Retrieving deals from Hubspot


[2025-03-06 18:41:10 - INFO] Retrieved 775 deals from Hubspot in 1.79 seconds
[2025-03-06 18:41:10 - INFO] Retrieving first calls from Hubspot
[2025-03-06 18:41:13 - INFO] Retrieved 1397 first calls from Hubspot in 2.9 seconds
/home/anton/Code/analytics_dashboard/hubspot_conversions.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  verbal_agreement_deals["conversion"] = "Verbal agreement after first call"
/home/anton/Code/analytics_dashboard/hubspot_conversions.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

FacebookRequestError: 

  Message: Call was not successful
  Method:  GET
  Path:    https://graph.facebook.com/v22.0/120211657450830005/insights
  Params:  {'date_preset': 'last_90d', 'time_increment': 1, 'breakdowns': 'campaign_name', 'fields': 'campaign_name,spend,impressions,clicks,reach'}

  Status:  400
  Response:
    {
      "error": {
        "message": "(#100) breakdowns[0] must be one of the following values: ad_format_asset, age, app_id, body_asset, breakdown_ad_objective, breakdown_reporting_ad_id, call_to_action_asset, coarse_conversion_value, country, description_asset, fidelity_type, gender, hsid, image_asset, impression_device, is_conversion_id_modeled, is_rendered_as_delayed_skip_ad, landing_destination, link_url_asset, mdsa_landing_destination, media_asset_url, media_creator, media_destination_url, media_format, media_origin_url, media_text_content, media_type, postback_sequence_index, product_id, redownload, region, skan_campaign_id, skan_conversion_id, skan_version

In [ ]:
# Ensure date column is datetime
conversions["date"] = pd.to_datetime(conversions["date"])

# Create widgets for filters and comparisons
date_range = pn.widgets.DateRangeSlider(
    name="Date Range",
    start=conversions["date"].min().date(),
    end=conversions["date"].max().date(),
    value=(conversions["date"].min().date(), conversions["date"].max().date()),
)

conversion_filter = pn.widgets.MultiChoice(
    name="Conversion Types",
    options=list(conversions["conversion"].unique()),
    value=list(conversions["conversion"].unique()),
)

time_agg = pn.widgets.RadioButtonGroup(
    name="Time Aggregation", options=["daily", "weekly", "monthly"], value="daily"
)

# UTM Parameter filters
utm_campaign_filter = pn.widgets.MultiChoice(
    name="UTM Campaign",
    options=list(conversions["utm_campaign"].dropna().unique()),
)

utm_source_filter = pn.widgets.MultiChoice(
    name="UTM Source",
    options=list(conversions["utm_source"].dropna().unique()),
)

utm_medium_filter = pn.widgets.MultiChoice(
    name="UTM Medium",
    options=list(conversions["utm_medium"].dropna().unique()),
)

# Comparison selector
comparison_selector = pn.widgets.Select(
    name="Compare By",
    options=[
        "None",
        "utm_campaign",
        "utm_source",
        "utm_medium",
        "landing_page_id",
        "country",
    ],
    value="None",
)